In [1]:

import h5py
import torch
import torch.nn as nn
import random


import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tempfile import TemporaryFile
from scipy.io import loadmat
from scipy.io import savemat
from torch.utils.data import DataLoader, TensorDataset

# Add the Torch_code directory to the Python path
# sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath('../helper'))
import config
import utils
import loader
# import skfuzzy as fuzz

In [3]:
FILE_PATH = os.path.dirname(os.path.abspath("__file__"))
print(FILE_PATH)

c:\Users\AT30890\Hoctap\Hprediction\H_est_cGAN\Channel_Estimation_cGAN_new\Channel_Estimation_cGAN\Torch_code\train


In [4]:
print(config.FILE_PATH)

c:\Users\AT30890\Hoctap\Hprediction\H_est_cGAN\Channel_Estimation_cGAN_new\Channel_Estimation_cGAN\Torch_code


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
device

device(type='cuda', index=0)

In [7]:
# Configuration
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

BATCH_SIZE = 32 #64  # Batch size
NUM_EPOCHS = 20


In [11]:

# rows from DeepMIMO dataset settings
# change rows according to the .mat dataset file 
rows = [['550', '568']] # , ['5100','5109']
rowss = "550_568"
SNR = np.arange(0, 36, 5) # 0:5:35 dB
outer_file_path = os.path.join(config.FILE_PATH, '..', 'DeepMIMOv2', 'Gan_Data', 'Static_612x14', 'freq_symb_1ant_612sub')
for snr in SNR:

    print(f" SNR: {snr}/{SNR[-1]}")

    H_true = np.empty((0, 2, 612, 14)) # true channel

    H_equal = np.empty((0, 2, 612, 14)) # noisy channel # LS channel
    H_linear = np.empty((0, 2, 612, 14)) # noisy channel # LS+Linear Interpolated channel
    H_practical = np.empty((0, 2, 612, 14)) # noisy channel # Practical Estimated channel

    # read data from ifferent .mat file, then concatenate them
    for i in range(len(rows)):
        file_path_partial = 'Gan_' + str(snr) +'_dBOutdoor1_60_1ant_612subcs_Row_' + rows[i][0] +'_' + rows[i][1] + '.mat'

        file_path = os.path.join(outer_file_path, file_path_partial)
        file_path = os.path.normpath(file_path)
        file = h5py.File(file_path, 'r')
        
        H_true = np.concatenate((H_true, np.array(file['H_data'])), axis = 0) # N_samples x channel(2) x height(614) x width(14)
        H_equal = np.concatenate((H_equal, np.array(file['H_equalized_data'])), axis = 0)
        H_linear = np.concatenate((H_linear, np.array(file['H_linear_data'])), axis=0)
        H_practical = np.concatenate((H_practical, np.array(file['H_practical_data'])), axis=0)

    shuffle_order = np.random.permutation(H_true.shape[0]);
    H_true = torch.tensor(H_true[shuffle_order])
    H_equal = torch.tensor(H_equal[shuffle_order])
    H_linear = torch.tensor(H_linear[shuffle_order])
    H_practical = torch.tensor(H_practical[shuffle_order])

    train_size = np.floor(H_practical.shape[0]*0.9) //BATCH_SIZE *BATCH_SIZE
    # print(train_size)
    # print(train_size/64)
    # print(train_size/input_data.size(0))
    train_size = int(train_size)

    # -----------------------------------------------------
    # 1. When input is H_linear (after LS+LI)
    print(f" Training for LS+LI")
    # [samples, 2, 612, 14]
    # 1.1 Split into training and validation sets for H_NN training
    trainData   = H_linear[0:train_size,:,:,:].to(device, dtype=torch.float)
    trainLabels = H_true[0:train_size,:,:,:].to(device, dtype=torch.float)

    valData   = H_linear[train_size:,:,:,:].to(device, dtype=torch.float)
    valLabels = H_true[train_size:,:,:,:].to(device, dtype=torch.float)

    # Split H_equal, H_linear, H_practical for validation later
    H_equal_val = H_equal[train_size:,:,:,:].to(device, dtype=torch.float)
    H_linear_val = H_linear[train_size:,:,:,:].to(device, dtype=torch.float)
    H_practical_val = H_practical[train_size:,:,:,:].to(device, dtype=torch.float)

    # 1.2 Normalization
    trainData_min = trainData.min()
    trainData_max = trainData.max()
    trainLabels_min = trainLabels.min()
    trainLabels_max = trainLabels.max()

    trainData_normd   = (trainData - trainData_min)/ (trainData_max - trainData_min)
    trainLabels_normd = (trainLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min)
    valData_normd     = (valData - trainData_min)/ (trainData_max - trainData_min)
    valLabels_normd   = (valLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min)
    # for evaluation, output of model(valData) will be de-normalized and compared with valLabels

    # Split real and imaginary grids into 2 image sets, then concatenate
    trainData_normd   = torch.cat((trainData_normd[:,0,:,:], trainData_normd[:,1,:,:]), dim=0).unsqueeze(1)  # 612 x 14 x (Nsamples*2)
    trainLabels_normd = torch.cat((trainLabels_normd[:,0,:,:], trainLabels_normd[:,1,:,:]), dim=0).unsqueeze(1)  # 612 x 14 x (Nsamples*2)

    # 1.3 Create a DataLoader for dataset
    dataset = TensorDataset(trainData_normd, trainLabels_normd)  # [4224, 1, 612, 14]
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    val_dataset = TensorDataset(valData_normd, valLabels_normd)  # [241, 2, 612, 14]
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 1.4 model
    model = utils.CNN_Est().to(device)

    learning_rate = 0.00001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    criterion = nn.MSELoss()

    # 1.5 Training loop
    train_loss =[]
    val_loss = []
    H_NN_val = torch.empty_like(valLabels) # [nVal, 2, 612, 14]
    num_epochs = NUM_EPOCHS
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        if (epoch == num_epochs-1):
            i = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        avg_train_loss = running_loss / len(train_loader)
        train_loss.append(avg_train_loss)
        print(f"SNR: {snr}/{SNR[-1]}, LS+LI, Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss} ")
        
        # Validation 
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                val_inputs_real = val_inputs[:,0,:,:].unsqueeze(1)
                val_inputs_imag = val_inputs[:,1,:,:].unsqueeze(1)
                val_targets_real = val_targets[:,0,:,:].unsqueeze(1)
                val_targets_imag = val_targets[:,1,:,:].unsqueeze(1)
                
                val_outputs_real = model(val_inputs_real)
                val_loss_real = criterion(val_outputs_real, val_targets_real)
                running_val_loss += val_loss_real.item()
                
                val_outputs_imag = model(val_inputs_imag)
                val_loss_imag = criterion(val_outputs_imag, val_targets_imag)
                running_val_loss += val_loss_imag.item()
                
                if (epoch == num_epochs-1):
                    H_NN_val[i:i+val_outputs_real.size(0),0,:,:].unsqueeze(1).copy_(val_outputs_real)
                    H_NN_val[i:i+val_outputs_imag.size(0),1,:,:].unsqueeze(1).copy_(val_outputs_imag)
                    i = i+val_outputs_imag.size(0)
                
        avg_val_loss = running_val_loss / (len(val_loader)*2)
        val_loss.append(avg_val_loss)    
                
        print(f"SNR: {snr}/{SNR[-1]}, LS+LI, Val Loss: {avg_val_loss}")


    save_folder = os.path.join(config.FILE_PATH, 'model/static/CNN', rowss, str(snr)+'dB')
    index_save = loader.find_incremental_filename(save_folder, 'CNN_', '_variable')

    model_save_path = os.path.join(save_folder,  'CNN_' +str(index_save)+'_LS_LI_CNN_model.pth')
    variable_save_path = os.path.join(save_folder, 'CNN_' +str(index_save)+'_variable.pth')
    params_save_path = os.path.join(save_folder, 'CNN_' +str(index_save)+'_params.mat')
    
    params = {   
                'SNR': snr,
                'epoc': NUM_EPOCHS,
                'rows': rowss,
                'learning_rate': learning_rate
    }
    savemat(params_save_path, params)
    variables = {             
                'train_track_LI': train_loss,
                'val_track_LI': val_loss,
                'train_min_LI': trainData_min.cpu(),
                'train_max_LI': trainData_max.cpu(),
                'val_min': trainLabels_min.cpu(),
                'val_max': trainLabels_max.cpu,
    }
    # variable to save 
    # Save the models' state dictionaries 
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
        }, model_save_path)


    plt.figure(figsize=(10, 5))
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(os.path.join(config.FILE_PATH, 'results', 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_LI_Loss.png') )
    plt.clf()


    # True channel
    H_val_true = valLabels.cpu()
    # convert to complex matrices
    H_val_true_complex = torch.complex(H_val_true[:,0,:,:], H_val_true[:,1,:,:])
    # variables['H_val_true'] = H_val_true # (nVal, 2, 612, 14)

    plt.figure(figsize=(10, 5))
    plt.imshow(H_val_true[-1,0,:,:],  aspect='auto', cmap='viridis', interpolation='none')
    plt.xlabel('OFDM symbol')
    plt.ylabel('Subcarrier')
    plt.title('True Channel')
    plt.colorbar()
    plt.savefig(os.path.join(config.FILE_PATH, 'results',  'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_trueChannel.png') )
    plt.clf()


    # Linear interpolated channel
    H_val_linInterp = valData.cpu()
    # convert to complex matrices
    H_val_linInterp_complex = torch.complex(H_val_linInterp[:,0,:,:], H_val_linInterp[:,1,:,:])

    # NMSE of Linear Interpolation
    # Calculate the mean squared error
    mse_LI = torch.mean(torch.abs(H_val_true_complex - H_val_linInterp_complex) ** 2)
    # Calculate the variance of the reference tensor (complex_tensor1)
    variance = torch.var(H_val_true_complex)
    # Calculate the NMSE
    nmse_LI = mse_LI / variance
    variables['NMSE_LI'] = nmse_LI.cpu()
    print(f"LS+LI NMSE: {nmse_LI.item()}")

    plt.figure(figsize=(10, 5))
    plt.imshow(H_val_linInterp[-1,0,:,:],  aspect='auto', cmap='viridis', interpolation='none')
    plt.xlabel('OFDM symbol')
    plt.ylabel('Subcarrier')
    plt.title(f'LS + Interpolate Estimated Channel, NMSE: {nmse_LI:.4f}')
    plt.colorbar()
    plt.savefig(os.path.join(config.FILE_PATH, 'results', 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_LI_estimatedChan.png') )
    # plt.show()
    plt.clf()

    # Estimated Channel 
    H_val_NN = H_NN_val.cpu()
    plt.figure(figsize=(10, 5))
    plt.imshow(H_val_NN[-1,0,:,:],  aspect='auto', cmap='viridis', interpolation='none')
    plt.xlabel('OFDM symbol')
    plt.ylabel('Subcarrier')
    plt.title('LI+CNN Estimated Channel (before de-normlized)')
    plt.colorbar()
    plt.savefig(os.path.join(config.FILE_PATH, "results", 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_LI_CNN_estimatedChan_before_denorm.png') )
    # plt.show()
    plt.clf()


    # De-normalized
    H_val_NN_denormd = H_NN_val * (trainLabels_max - trainLabels_min) + trainLabels_min
    H_val_NN_denormd = H_val_NN_denormd.cpu()

    # variables['H_val_LI_NN'] = H_val_NN_denormd # (nVal, 2, 612, 14)

    # convert to complex matrices
    H_val_NN_denormd_complex = torch.complex(H_val_NN_denormd[:,0,:,:], H_val_NN_denormd[:,1,:,:])

    # NMSE of Linear Interpolation + NN
    # Calculate the mean squared error
    mse_LI_NN = torch.mean(torch.abs(H_val_true_complex - H_val_NN_denormd_complex) ** 2)
    # Calculate the NMSE
    nmse_LI_NN = mse_LI_NN / variance
    print(f"LS+LI+CNN NMSE: {nmse_LI_NN.item()}")
    variables['NMSE_LI_NN'] = nmse_LI_NN.cpu()

    plt.figure(figsize=(10, 5))
    plt.imshow(H_val_NN_denormd[-1,0,:,:],  aspect='auto', cmap='viridis', interpolation='none')
    plt.xlabel('OFDM symbol')
    plt.ylabel('Subcarrier')
    plt.title(f'LI+CNN Estimated Channel (after de-normlized), NMSE: {nmse_LI_NN:.4f}')
    plt.colorbar()
    plt.savefig(os.path.join(config.FILE_PATH, 'results', 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_LI_CNN_estimatedChan.png') )
    plt.clf()



    # ------------------------------------------------------
    # When Input of the NN is just H_equalized
    print(f" Training for LS")
    # [samples, 2, 612, 14]
    # Split into training and validation sets for H_NN training
    trainData   = H_equal[0:train_size,:,:,:].to(device, dtype=torch.float)
    trainLabels = H_true[0:train_size,:,:,:].to(device, dtype=torch.float)

    valData   = H_equal[train_size:,:,:,:].to(device, dtype=torch.float)
    valLabels = H_true[train_size:,:,:,:].to(device, dtype=torch.float)

    # Normalization
    trainData_min = trainData.min()
    trainData_max = trainData.max()
    trainLabels_min = trainLabels.min()
    trainLabels_max = trainLabels.max()

    trainData_normd   = (trainData - trainData_min)/ (trainData_max - trainData_min)
    trainLabels_normd = (trainLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min)
    valData_normd     = (valData - trainData_min)/ (trainData_max - trainData_min)
    valLabels_normd   = (valLabels - trainLabels_min)/ (trainLabels_max - trainLabels_min)


    # Split real and imaginary grids into 2 image sets, then concatenate
    trainData_normd   = torch.cat((trainData_normd[:,0,:,:], trainData_normd[:,1,:,:]), dim=0).unsqueeze(1)  # 612 x 14 x (Nsamples*2)
    trainLabels_normd = torch.cat((trainLabels_normd[:,0,:,:], trainLabels_normd[:,1,:,:]), dim=0).unsqueeze(1)  # 612 x 14 x (Nsamples*2)

    H_temp = trainData.cpu()
    plt.figure(figsize=(10, 5))
    plt.imshow(H_temp[0,0,:,:],  aspect='auto', cmap='viridis', interpolation='none')
    plt.colorbar()
    plt.xlabel('OFDM symbol')
    plt.ylabel('Subcarrier')
    plt.title(f'LS Channel')
    plt.savefig(os.path.join(config.FILE_PATH, 'results', 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_Chan.png') )
    plt.clf()


    # Create a DataLoader for dataset
    dataset = TensorDataset(trainData_normd, trainLabels_normd)  # [4224, 1, 612, 14]
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    val_dataset = TensorDataset(valData_normd, valLabels_normd)  # [241, 2, 612, 14]
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model2 = utils.CNN_Est().to(device)
    learning_rate = 0.00001
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate) 
    criterion = nn.MSELoss()

    # Training loop
    train_loss =[]
    val_loss = []
    H_NN_val = torch.empty_like(valLabels) # [nVal, 2, 612, 14]
    num_epochs = NUM_EPOCHS
    for epoch in range(num_epochs):
        model2.train()
        running_loss = 0.0
        if (epoch == num_epochs-1):
            i = 0
        for inputs, targets in train_loader:
            optimizer2.zero_grad()
            outputs = model2(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer2.step()
            running_loss += loss.item()
            
        avg_train_loss = running_loss / len(train_loader)
        train_loss.append(avg_train_loss)
        print(f"SNR: {snr}/{SNR[-1]}, LS, Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss} ")
        
        # Validation 
        model2.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                val_inputs_real = val_inputs[:,0,:,:].unsqueeze(1)
                val_inputs_imag = val_inputs[:,1,:,:].unsqueeze(1)
                val_targets_real = val_targets[:,0,:,:].unsqueeze(1)
                val_targets_imag = val_targets[:,1,:,:].unsqueeze(1)
                
                val_outputs_real = model2(val_inputs_real)
                val_loss_real = criterion(val_outputs_real, val_targets_real)
                running_val_loss += val_loss_real.item()
                
                val_outputs_imag = model2(val_inputs_imag)
                val_loss_imag = criterion(val_outputs_imag, val_targets_imag)
                running_val_loss += val_loss_imag.item()
                
                if (epoch == num_epochs-1):
                    H_NN_val[i:i+val_outputs_real.size(0),0,:,:].unsqueeze(1).copy_(val_outputs_real)
                    H_NN_val[i:i+val_outputs_imag.size(0),1,:,:].unsqueeze(1).copy_(val_outputs_imag)
                    i = i+val_outputs_imag.size(0)
                
        avg_val_loss = running_val_loss / (len(val_loader)*2)
        val_loss.append(avg_val_loss)    
                
        print(f"SNR: {snr}/{SNR[-1]}, LS, Val Loss: {avg_val_loss}")

    plt.figure(figsize=(10, 5))
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(os.path.join(config.FILE_PATH, 'results', 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_Loss.png') )
    plt.clf()

    H_val_NN_denormd = H_NN_val * (trainLabels_max - trainLabels_min) + trainLabels_min
    H_val_NN_denormd = H_val_NN_denormd.cpu()

    model_save_path = os.path.join(save_folder,  'CNN_' +str(index_save)+'_LS_CNN_model.pth')

    # variables['H_val_LS_NN']= H_val_NN_denormd.cpu() # (nVal, 2, 612, 14)
    variables['train_track_LS']= train_loss
    variables['val_track_LS']= val_loss
    variables['train_min_LS']= trainData_min.cpu()
    variables['train_max_LS']= trainData_max.cpu()

    # variable to save 
    # Save the models' state dictionaries 
    torch.save({'model_state_dict': model2.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, model_save_path)


    # NMSE of LS + NN
    H_val_LS_NN_complex = torch.complex(H_val_NN_denormd[:,0,:,:], H_val_NN_denormd[:,1,:,:])
    # Calculate the mean squared error
    mse_LS_NN = torch.mean(torch.abs(H_val_true_complex - H_val_LS_NN_complex) ** 2)
    # Calculate the NMSE
    nmse_LS_NN = mse_LS_NN / variance
    print(f"LS+CNN NMSE: {nmse_LS_NN.item()}")
    variables['NMSE_LS_NN'] = nmse_LS_NN.cpu()

    plt.figure(figsize=(10, 5))
    plt.imshow(H_val_NN_denormd[-1,0,:,:],  aspect='auto', cmap='viridis', interpolation='none')
    plt.xlabel('OFDM symbol')
    plt.ylabel('Subcarrier')
    plt.title(f'LS+CNN Estimated Channel (after de-normlized), NMSE: {nmse_LS_NN:.4f}')
    plt.colorbar()
    plt.savefig(os.path.join(config.FILE_PATH, 'results', 'static', 'CNN', rowss, str(snr) + 'dB',  str(index_save) + '_LS_CNN_estimatedChan.png') )
    plt.clf()

    torch.save( variables,variable_save_path)

 SNR: 0/35
 Training for LS+LI
SNR: 0/35, LS+LI, Epoch 1/20, Loss: 0.09429796080803499 
SNR: 0/35, LS+LI, Val Loss: 0.028029144353543717
SNR: 0/35, LS+LI, Epoch 2/20, Loss: 0.01724621262110304 
SNR: 0/35, LS+LI, Val Loss: 0.01219334825873375
SNR: 0/35, LS+LI, Epoch 3/20, Loss: 0.007287797718163347 
SNR: 0/35, LS+LI, Val Loss: 0.002619391850506266
SNR: 0/35, LS+LI, Epoch 4/20, Loss: 0.001714134645832625 
SNR: 0/35, LS+LI, Val Loss: 0.0013193897563420858
SNR: 0/35, LS+LI, Epoch 5/20, Loss: 0.0010574158350209473 
SNR: 0/35, LS+LI, Val Loss: 0.000882947351783514
SNR: 0/35, LS+LI, Epoch 6/20, Loss: 0.0007446678804020243 
SNR: 0/35, LS+LI, Val Loss: 0.0006414688056490073
SNR: 0/35, LS+LI, Epoch 7/20, Loss: 0.0005508470918963818 
SNR: 0/35, LS+LI, Val Loss: 0.0005020009254318817
SNR: 0/35, LS+LI, Epoch 8/20, Loss: 0.00043238663389881066 
SNR: 0/35, LS+LI, Val Loss: 0.0003962052602825376
SNR: 0/35, LS+LI, Epoch 9/20, Loss: 0.00036057899327109527 
SNR: 0/35, LS+LI, Val Loss: 0.00033969005016842

C:\Users\AT30890\AppData\Local\Temp\ipykernel_23068\1234052587.py:251: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 5))


 Training for LS
SNR: 10/35, LS, Epoch 1/20, Loss: 0.1214221907236303 
SNR: 10/35, LS, Val Loss: 0.03825410036370158
SNR: 10/35, LS, Epoch 2/20, Loss: 0.029097594403234932 
SNR: 10/35, LS, Val Loss: 0.02281663039078315
SNR: 10/35, LS, Epoch 3/20, Loss: 0.020556683302856982 
SNR: 10/35, LS, Val Loss: 0.016445850487798452
SNR: 10/35, LS, Epoch 4/20, Loss: 0.00963937026729885 
SNR: 10/35, LS, Val Loss: 0.0026196724987433604
SNR: 10/35, LS, Epoch 5/20, Loss: 0.0014330238755064784 
SNR: 10/35, LS, Val Loss: 0.0009085535032985111
SNR: 10/35, LS, Epoch 6/20, Loss: 0.0007254838421128321 
SNR: 10/35, LS, Val Loss: 0.000544130826407733
SNR: 10/35, LS, Epoch 7/20, Loss: 0.0004540430786619254 
SNR: 10/35, LS, Val Loss: 0.00036060816880005103
SNR: 10/35, LS, Epoch 8/20, Loss: 0.0002947949856206833 
SNR: 10/35, LS, Val Loss: 0.000237406199327476
SNR: 10/35, LS, Epoch 9/20, Loss: 0.0002221192296474328 
SNR: 10/35, LS, Val Loss: 0.00019967994315569135
SNR: 10/35, LS, Epoch 10/20, Loss: 0.0001922541843

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [ ]:
print(variables)

{'SNR': 0, 'epoc': 20, 'rows': '550_568', 'learning_rate': 1e-05, 'train_track_LI': [0.12542229983955622, 0.016818697966906864, 0.006570089958889487, 0.0017958001614412449, 0.0010687704604303387, 0.0006360203134742429, 0.0004226578427430165, 0.000339401617641973, 0.00029624920997169585, 0.00026810644628009567, 0.0002501570282523365, 0.00023476741269708631, 0.0002248910150986679, 0.00021715459956794803, 0.00021035958222152354, 0.00020660342367288345, 0.00019989069134377738, 0.00019464028150650847, 0.00019122100115964713, 0.00018718416041944388], 'val_track_LI': [0.028704328772922356, 0.01160938641987741, 0.0027627505478449166, 0.0013673651604525123, 0.0008469064535650735, 0.0005033311208535451, 0.0003792883859811506, 0.00032053827332371537, 0.00029038065440545324, 0.00026359001640230417, 0.00025747243004540604, 0.00023620072109527732, 0.0002252046312302506, 0.00022641037867288105, 0.0002112559110779936, 0.0002063743898664446, 0.00020225443404342514, 0.00019736855877757384, 0.00019304889